**Imports**

In [2]:
import os
import psycopg2

In [5]:
mot_passe = os.environ.get('pg_psw')

print(mot_passe)

Th302k18


**Connection + création DB**

In [7]:
conn = psycopg2.connect(
   database="postgres", user='postgres', password=mot_passe, host='localhost', port= '5432'
)
conn.autocommit = True

cursor = conn.cursor()

sql = '''CREATE database acde_manager''';

cursor.execute(sql)
print("Base de données créée avec succès !")

conn.close()

Base de données créée avec succès !


**Fonctions génériques**

In [11]:
def ouvrir_connection(nom_bdd, utilisateur, mot_passe, host='localhost', port=5432):
    try:
        conn = psycopg2.connect(dbname=nom_bdd, user=utilisateur, password=mot_passe, host=host, port=5432)
    except psycopg2.Error as e:
        print("Erreur lors de la connection à la base de données")
        print(e)
        return None
    # On force autocommit (non applicable ds SQLite3)
    conn.set_session(autocommit=True)
    return conn


def supprimer_table(conn, sql_suppression_table):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_suppression_table)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la suppression de la table")
        print(e)
        return
    cursor.close()
    print("La table a été supprimée avec succès")

    
def creer_table(conn, sql_creation_table):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_creation_table)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la création de la table")
        print(e)
        return
    cursor.close()
    print("La table a été crée avec succès")

    
def inserer_donnees(conn, sql_insertion_table):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_insertion_table)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de l'insertion des données")
        print(e)
        return
    cursor.close()
    print("Les données ont été insérées avec succès")

    
def lire_donnees(conn, sql_requete):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_requete)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la lecture des données")
        print(e)
        return None
    
    print("Les données ont été lues avec succès")
    data = []
    for row in cursor:
        data.append(row)

    cursor.close()
    
    return data

**Requêtes SQL pour la table manager**

In [37]:
sql_supprimer_table_manager = """
    DROP TABLE IF EXISTS ;
"""

sql_creer_table_manager = """
    CREATE TABLE IF NOT EXISTS manager (
    id integer primary key,
    equipe_id integer,
    type text,
    firstname text,
    lastname text,
    email text,
    identifiant text,
    password text,
    admin_client integer,
    admin_dev integer,
    visiteur integer
    );
"""

sql_inserer_manager = """
    INSERT INTO manager 
    (id, equipe_id, type, firstname, lastname, email, identifiant,
    password, admin_client, admin_dev, visiteur)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

sql_inserer_manager = """
    COPY manager 
    (id, equipe_id, type, firstname, lastname, email, identifiant,
    password, admin_client, admin_dev, visiteur)
    FROM 'C:\\Users\\Public\\Documents\\acde_manager.csv'
    WITH CSV DELIMITER ','
    QUOTE '"'
    HEADER;
"""

sql_inserer_manager_dict = """
    INSERT INTO manager 
    (id, equipe_id, type, firstname, lastname, email, identifiant,
    password, admin_client, admin_dev, visiteur)
    VALUES (%(id)s, %(equipe_id)s, %(type)s, %(firstname)s, %(lastname)s, %(email)s, %(identifiant)s,
    %(password)s, %(admin_client)s, %(admin_dev)s, %(visiteur)s);
"""

sql_lire_manager = """
    SELECT *
    FROM manager;
"""

**parametres bdd**

In [28]:
ma_base_donnees = "acde_manager"
utilisateur = "postgres"
mot_passe = os.environ.get('pg_psw')

**Connect BDD**

In [29]:
conn = ouvrir_connection(ma_base_donnees, utilisateur, mot_passe)

type(conn)

psycopg2.extensions.connection

**Create tables**

In [30]:
creer_table(conn, sql_creer_table_manager)

La table a été crée avec succès


**import CSV**

In [42]:
inserer_donnees(conn, sql_inserer_manager)

Erreur lors de l'insertion des données
ERREUR:  séquence d'octets invalide pour l'encodage « UTF8 » : 0xe9 0x6e 0x6f
CONTEXT:  COPY manager, ligne 2



**Lecture data**

In [41]:
lire_donnees(conn, sql_lire_manager)

Les données ont été lues avec succès


[]

**Delete tables**

In [ ]:
supprimer_table(conn, sql_supprimer_table_manager)